In [1]:
# examining data files in source
import numpy as np
import pandas as pd
import pickle 
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

/home1/efeghhi/.conda/envs/env1/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):


In [2]:
session_events_amy = pd.DataFrame()
session_events_hpc = pd.DataFrame()
ripple_array_amy = []
ripple_array_hpc = []
sub_sess_names_amy = []
sub_sess_names_hpc = []
sub_names_amy = []
sub_names_hpc = []
trial_nums_amy = []
trial_nums_hpc = []
elec_ripple_rate_array_amy = []
elec_ripple_rate_array_hpc = []

In [3]:
fn_amy = '/scratch/john/SWR_scratch/ENCODING/SWR_catFR1_R1015J_0_AMY_encoding_soz_in_hamming.p'
with open(fn_amy,'rb') as f:
    dat = pickle.load(f)
    session_events_amy = session_events_amy.append(dat['session_events'])
    ripple_array_amy = superVstack(ripple_array_amy,dat['ripple_array'])
    sub_sess_names_amy.extend(dat['sub_sess_names'])
    sub_names_amy.extend(dat['sub_names'])
    trial_nums_amy = np.append(trial_nums_amy,dat['trial_nums'])
    elec_ripple_rate_array_amy.extend(dat['elec_ripple_rate_array']) 
    regions_amy = dat['HPC_names']

    fn_hpc = '/scratch/john/SWR_scratch/ENCODING/SWR_catFR1_R1015J_0_HPC_encoding_soz_in_hamming.p'
    with open(fn_hpc,'rb') as f:
        dat = pickle.load(f)
        session_events_hpc = session_events_hpc.append(dat['session_events'])
        ripple_array_hpc = superVstack(ripple_array_hpc,dat['ripple_array'])
        print(dat['ripple_array'].shape)
        sub_sess_names_hpc.extend(dat['sub_sess_names'])
        sub_names_hpc.extend(dat['sub_names'])
        trial_nums_hpc = np.append(trial_nums_hpc,dat['trial_nums'])
        elec_ripple_rate_array_hpc.extend(dat['elec_ripple_rate_array'])
        regions_hpc = dat['HPC_names']

(840, 1500)


In [15]:
def check_ca1_dg_ripples(ripple_aerray, hpc_regions, hpc_trialnums):

    lhd_ca1 = []
    lhd_dg = []
    rhd_ca1 = []
    rhd_dg = []

    trialStart = 0 
    
    for region, trials in zip(hpc_regions, hpc_trialnums):

        trials = int(trials)

        if 'right ca1' in region:
            rhd_ca1.append(ripple_array_hpc[trialStart:trialStart+trials])
        if 'right dg' in region:
            rhd_dg.append(ripple_array_hpc[trialStart:trialStart+trials])
        if 'left ca1' in region:
            lhd_ca1.append(ripple_array_hpc[trialStart:trialStart+trials])
        if 'left dg' in region:
            lhd_dg.append(ripple_array_hpc[trialStart:trialStart+trials])

        trialStart += trials 

    
    print(np.sum(rhd_ca1))
    print(np.sum(rhd_dg))
    print(np.sum(lhd_ca1))
    print(np.sum(lhd_dg))

In [16]:
check_ca1_dg_ripples(ripple_array_hpc, regions_hpc, trial_nums_hpc)

0.0
0.0
5962.5
1154.5


In [25]:
print(f"AMY ripple array shape: {np.array(ripple_array_amy).shape}")
print(f"HPC ripple array shape: {np.array(ripple_array_hpc).shape}")

print(f"AMY sub names: {sub_names_amy}")
print(f"HPC sub names: {sub_names_hpc}")

print(f"AMY sub names: {sub_names_amy}")
print(f"HPC sub names: {sub_names_hpc}")

print(f"AMY sub_sess names: {sub_sess_names_amy}")
print(f"HPC sub_sess names: {sub_sess_names_hpc}")

print(f"AMY trial nums: {trial_nums_amy}")
print(f"HPC trial nums: {trial_nums_hpc}")

AMY ripple array shape: (240, 1500)
HPC ripple array shape: (840, 1500)
AMY sub names: ['R1015J', 'R1015J']
HPC sub names: ['R1015J', 'R1015J', 'R1015J', 'R1015J', 'R1015J', 'R1015J', 'R1015J']
AMY sub names: ['R1015J', 'R1015J']
HPC sub names: ['R1015J', 'R1015J', 'R1015J', 'R1015J', 'R1015J', 'R1015J', 'R1015J']
AMY sub_sess names: ['R1015J-0', 'R1015J-0']
HPC sub_sess names: ['R1015J-0', 'R1015J-0', 'R1015J-0', 'R1015J-0', 'R1015J-0', 'R1015J-0', 'R1015J-0']
AMY trial nums: [120. 120.]
HPC trial nums: [120. 120. 120. 120. 120. 120. 120.]


In [26]:
np.sum(trial_nums_amy)

240.0

In [17]:
def ripples_hpc(nonhpc_trialnums, nonhpc_regions, hpc_trialnums, hpc_regions, ripple_array_hpc, regions_selected):

    
    timesteps = ripple_array_hpc.shape[1]
    num_trials = np.sum(nonhpc_trialnums)

    # Step 1: Separate hpc right and left ripples
    left_hemi_data = []
    right_hemi_data = []
    trialStart = 0
    for region, trials in zip(hpc_regions, hpc_trialnums):
        trials = int(trials)
        for rs in regions_selected:
            if f'right {rs}' in region: 
                right_hemi_data.append(ripple_array_hpc[trialStart:trialStart+trials])
            if f'left {rs}' in region: 
                left_hemi_data.append(ripple_array_hpc[trialStart:trialStart+trials])
        trialStart += trials 

    if len(right_hemi_data) > 0:
        rhd_np = np.clip(np.sum(np.stack(right_hemi_data),axis=0), a_min=0, a_max=1)
    else:
        rhd_np = np.zeros_like((trials, timesteps))
    if len(left_hemi_data) > 0:
        lhd_np = np.clip(np.sum(np.stack(left_hemi_data),axis=0), a_min=0, a_max=1)
    else:
        lhd_np = np.zeros_like((trials, timesteps))


    ripples_hpc_nonhpcshape = np.zeros((int(num_trials), timesteps))

    trialStart = 0
    for region, trials in zip(nonhpc_regions, nonhpc_trialnums):
        trials = int(trials)
        
        if 'right' in region:
            ripples_hpc_nonhpcshape[trialStart:trialStart+trials] = rhd_np
        if 'left' in region:
            ripples_hpc_nonhpcshape[trialStart:trialStart+trials] = lhd_np

        trialStart += trials 

    return ripples_hpc_nonhpcshape



In [24]:
ripples_hpc_amyshape = ripples_hpc(trial_nums_amy, regions_amy,
                                 trial_nums_hpc, regions_hpc, ripple_array_hpc, ['ca1'])

In [27]:
np.where(np.sum(ripples_hpc_amyshape, axis=1)==0, x>=1


(240,)